# **Setting Up the Environment**

## Library Import

In [1]:
import pandas as pd
import numpy as np
import sklearn

## Data Import

In [2]:
df = pd.read_csv('./nyc-taxi-trip-duration/train.csv')
df.shape

(1458644, 11)

In [3]:
# Sample a random 10 rows
df.sample(10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1275084,id1316218,1,2016-06-17 04:02:28,2016-06-17 04:14:18,1,-73.987709,40.721054,-73.954971,40.745071,N,710
175063,id2752282,2,2016-06-15 06:52:23,2016-06-15 07:08:02,1,-73.990303,40.771835,-74.013542,40.715569,N,939
1011320,id2449794,1,2016-01-17 20:29:36,2016-01-17 20:39:08,1,-73.862442,40.770126,-73.921120,40.774483,N,572
605947,id0798317,1,2016-02-06 18:39:16,2016-02-06 18:53:16,3,-73.986832,40.768494,-73.987999,40.750748,N,840
741247,id0091968,1,2016-03-09 20:40:44,2016-03-09 21:01:59,2,-73.954933,40.773376,-73.996521,40.727024,N,1275
1125815,id1215018,1,2016-03-20 19:00:59,2016-03-20 19:14:06,1,-74.000900,40.720669,-74.005089,40.735939,N,787
212015,id3399275,1,2016-06-27 08:36:27,2016-06-27 08:42:25,1,-73.969810,40.756878,-73.955315,40.764648,N,358
1029101,id1216442,2,2016-04-23 17:13:20,2016-04-23 17:25:28,1,-73.978012,40.758591,-73.984802,40.760529,N,728
1428182,id1047230,1,2016-01-15 11:35:11,2016-01-15 11:36:40,1,-73.995285,40.744831,-73.999329,40.738987,N,89
1264061,id3519038,1,2016-02-10 13:46:40,2016-02-10 13:49:33,1,-74.003838,40.726406,-74.004524,40.732712,N,173


## Artificial Data Generation
To simulate dirty data, we will add more records to the data such that
- there will be missing values
- some of the values might not be in the right format or data type such as a number stored as a string
- there are duplicates in the data
- outliers exist
- non-normalized values (i.e. values that are not in the same scale) or non-standardized values (i.e. values that are not in the same range)
- multi-collinearity

### Missing Values

In [4]:
from tqdm import tqdm
import time

# Add new records which is 10% of the original dataset but with missing values
def simulateMissingVals(df, pctEmpty=10):
    num_missing_rows = int(df.shape[0] * pctEmpty / 100)
    
    # Create the new records
    new_records = df.sample(num_missing_rows, replace=False)
    
    # Randomly select one/more of the columns to set as np.nan
    for i in tqdm(new_records.index, desc="Progress", unit="item"):
        num_empty_cols = np.random.randint(1, df.shape[1]+1)
        empty_cols = np.random.choice(df.columns, num_empty_cols, replace=False)
        new_records.loc[i, empty_cols] = np.nan
    
    # Append the new records to the original dataframe and place them randomly in the new dataframe
    df = df.append(new_records)
    df = df.sample(frac=1).reset_index(drop=True)
    
    return df

In [5]:
df_dirty = simulateMissingVals(df, pctEmpty=15)

df.shape, df_dirty.shape

Progress: 100%|██████████| 218796/218796 [09:36<00:00, 379.38item/s]
/var/folders/3k/8x521n3x1hv89v96qvzcfzb00000gn/T/ipykernel_2693/2312774374.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_records)


((1458644, 11), (1677440, 11))

In [6]:
# Save the dirty dataset
df_dirty.to_csv('./nyc-taxi-trip-duration/train_dirty.csv', index=False)

In [8]:
# Read in the latest dirty dataset
df_dirty = pd.read_csv('./nyc-taxi-trip-duration/train_dirty.csv')

df_dirty.isnull().sum()

id                    119576
vendor_id             119548
pickup_datetime       119744
dropoff_datetime      119455
passenger_count       119274
pickup_longitude      119424
pickup_latitude       119951
dropoff_longitude     119995
dropoff_latitude      119616
store_and_fwd_flag    119431
trip_duration         119676
dtype: int64

In [9]:
df_dirty.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.557892e+06,1.558166e+06,1.558016e+06,1.557489e+06,1.557445e+06,1.557824e+06,1.557764e+06
mean,1.534895e+00,1.664459e+00,-7.397349e+01,4.075092e+01,-7.397340e+01,4.075181e+01,9.591557e+02
std,4.987810e-01,1.313983e+00,6.935497e-02,3.266136e-02,6.914012e-02,3.600310e-02,5.130668e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073736e+01,-7.399133e+01,4.073589e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075409e+01,-7.397974e+01,4.075453e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076835e+01,-7.396300e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


### Artificial Categorical Variables

In [10]:
# Check the datatypes of each column
df_dirty.dtypes

id                     object
vendor_id             float64
pickup_datetime        object
dropoff_datetime       object
passenger_count       float64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration         float64
dtype: object

In [14]:
# Extract categorical variables
cat_vars = [col for col in df_dirty.columns if df_dirty[col].dtype == 'object']
print(f'The categorical variables are: {cat_vars}')

# Check the unique values for the CATEGORICAL VARIABLES
df_dirty['store_and_fwd_flag'].unique()

The categorical variables are: ['id', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag']


array(['N', nan, 'Y'], dtype=object)

Since there are **very few** CATEGORICAL VARIABLES in the dataset, let's artificaially create some categorical variables relevant to the context of the dataset: taxi trip dataset in NYC. Some examples of categorical variables that we can create are:
- boroughs
- neighborhoods
- taxi types
- payment types
- weather conditions

In [15]:
# Create some new CATEGORICAL VARIABLES: NYC borough, taxi types, and payment types
NYC_BOROUGHS = ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']
TAXI_TYPES = ['yellow', 'green', 'fhv']
PAYMENT_TYPES = ['cash', 'credit', 'no charge', 'dispute', 'unknown', 'voided trip']

# Create the categorical columns
df_dirty['nyc_borough'] = np.random.choice(NYC_BOROUGHS, df_dirty.shape[0])
df_dirty['taxi_type'] = np.random.choice(TAXI_TYPES, df_dirty.shape[0])
df_dirty['payment_type'] = np.random.choice(PAYMENT_TYPES, df_dirty.shape[0])

# Save the dirty dataset
df_dirty.to_csv('./nyc-taxi-trip-duration/train_dirty.csv', index=False)

In [16]:
# Reload the dirty dataset
df_dirty = pd.read_csv('./nyc-taxi-trip-duration/train_dirty.csv')

df_dirty.isnull().sum()

id                    119576
vendor_id             119548
pickup_datetime       119744
dropoff_datetime      119455
passenger_count       119274
pickup_longitude      119424
pickup_latitude       119951
dropoff_longitude     119995
dropoff_latitude      119616
store_and_fwd_flag    119431
trip_duration         119676
nyc_borough                0
taxi_type                  0
payment_type               0
dtype: int64

In [17]:
df_dirty.dtypes

id                     object
vendor_id             float64
pickup_datetime        object
dropoff_datetime       object
passenger_count       float64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration         float64
nyc_borough            object
taxi_type              object
payment_type           object
dtype: object

In [18]:
# For each of the three columns, randomly select a random number of rows to set as np.nan
for col in ['nyc_borough', 'taxi_type', 'payment_type']:
    num_missing_rows = np.random.randint(1, df_dirty.shape[0]+1)
    missing_rows = np.random.choice(df_dirty.index, num_missing_rows, replace=False)
    df_dirty.loc[missing_rows, col] = np.nan
    
# Save the dirty dataset
df_dirty.to_csv('./nyc-taxi-trip-duration/train_dirty.csv', index=False)

df_dirty.isnull().sum()

id                    119576
vendor_id             119548
pickup_datetime       119744
dropoff_datetime      119455
passenger_count       119274
pickup_longitude      119424
pickup_latitude       119951
dropoff_longitude     119995
dropoff_latitude      119616
store_and_fwd_flag    119431
trip_duration         119676
nyc_borough           561366
taxi_type             405660
payment_type          700956
dtype: int64

### Duplicates and Outliers

Let's create random duplicates and outliers in the dataset, to simulate dirty data.

In [2]:
df_dirty = pd.read_csv('./nyc-taxi-trip-duration/train_dirty.csv')
df_dirty.shape

(1677440, 14)

In [3]:
def duplicateRecords(df):
    print(f'Original: {df.shape[0]:,}')
    
    # Create a new dataframe with duplicate records
    random_pct = np.random.random() # Duplicate less than 1% of the original dataset
    print(f'Duplicating {random_pct:,.2}% of the original dataset ...')
    df_dup = df.sample(frac=random_pct/100, replace=True)
    
    # Append the new dataframe to the original dataframe
    df = pd.concat([df, df_dup], axis=0)
    
    # Shuffle the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    
    print(f'After duplication: {df.shape[0]:,}')
    
    return df

In [4]:
df_dirty = duplicateRecords(df_dirty)

Original: 1,677,440
Duplicating 0.66% of the original dataset ...
After duplication: 1,688,528


In [5]:
# Check the number of duplicate records
df_dirty.duplicated().sum()

35697

In [7]:
# Add outliers to the dataset
def addOutliers(df):
    print(f'Original: {df.shape[0]:,}')
    
    # Create a new dataframe with outliers
    random_pct = np.random.random() # Add outliers to less than 1% of the original dataset
    print(f'Adding outliers to {random_pct:,.2}% of the original dataset ...')
    
    # Randomly select records from the dataset
    df_outliers = df.sample(frac=random_pct/100, replace=True)
    
    # Replace the values of the selected records with outliers for the following columns
    for col in ['passenger_count', 'trip_duration']:
        df_outliers[col] = df_outliers[col] * np.random.randint(1000, 10_000)
    
    # Append the new dataframe to the original dataframe
    df = pd.concat([df, df_outliers], axis=0)
    
    # Shuffle the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    
    print(f'After adding outliers: {df.shape[0]:,}')
    
    return df

In [8]:
# Simulate outliers
df_dirty = addOutliers(df_dirty)

# Save the dirty dataset
df_dirty.to_csv('./nyc-taxi-trip-duration/train_dirty.csv', index=False)

Original: 1,688,528
Adding outliers to 0.11% of the original dataset ...
After adding outliers: 1,690,302


In [9]:
df_dirty[['passenger_count', 'trip_duration']].describe()

,passenger_count,trip_duration
count,1.570180e+06,1.569723e+06
mean,1.121235e+01,5.823368e+03
std,3.720397e+02,4.270312e+05
min,0.000000e+00,1.000000e+00
25%,1.000000e+00,3.970000e+02
50%,1.000000e+00,6.630000e+02
75%,2.000000e+00,1.077000e+03
max,3.252600e+04,4.274672e+08


### Non-Normalized and Non-Standardized Values

No need to artifically simulate this. Note that
- normalization - useful when a feature has a strict upper and lower bound, while
- standardization - useful when a feature has no strict upper and lower bound but is assumed to be normally distributed (i.e. Gaussian distribution)

In our existing dataset,
- `passenger_count` has a strict upper bound of 3 passengers as that's the only number of passengers that can fit in a sedan taxi
- `trip_distance` has no strict upper and lower bound but is assumed to be normally distributed

### Multi-collinearity

This is an often overlooked problem in data science. Multi-collinearity is when two or more features are highly correlated with each other. This is a problem because it makes it difficult for the model to distinguish between the effects of the two features on the target variable. In other words, the model will not be able to tell which feature is causing the change in the target variable.

In order to simulate this, let's create a new feature `trip_distance_km` that is the same as `trip_distance` but in kilometers instead of miles. This will create a perfect correlation between the two features.

In [10]:
df_dirty = pd.read_csv("./nyc-taxi-trip-duration/train_dirty.csv")
df_dirty.shape

(1690302, 14)

In [11]:
df_dirty.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'nyc_borough', 'taxi_type', 'payment_type'],
      dtype='object')

In [12]:
# Calculate the distance between the pickup and dropoff locations
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371 # Radius of the Earth in km
    
    # Convert latitude and longitude to radians
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi       = np.radians(lat2 - lat1)
    dlambda    = np.radians(lon2 - lon1)
    
    a = np.sin(dphi/2)**2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2)**2
    
    return 2*r*np.arctan2(np.sqrt(a), np.sqrt(1 - a))

In [13]:
# Calculate the distance between the pickup and dropoff locations
df_dirty['distance'] = haversine_distance(df_dirty['pickup_latitude'], df_dirty['pickup_longitude'], df_dirty['dropoff_latitude'], df_dirty['dropoff_longitude'])

df_dirty['speed'] = df_dirty['distance'] / df_dirty['trip_duration'] * 3600

df_dirty.shape

(1690302, 16)

In [15]:
df_dirty.sample(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,nyc_borough,taxi_type,payment_type,distance,speed
1115363,id2753458,2.0,2016-03-30 20:27:47,2016-03-30 21:01:37,1.0,-74.005203,40.728809,-73.960663,40.659550,N,2030.0,Queens,NaN,NaN,8.568035,15.194545
650989,id2556201,2.0,2016-01-12 05:21:18,2016-01-12 05:40:15,1.0,-73.992889,40.747791,-73.942833,40.702301,N,1137.0,Brooklyn,NaN,voided trip,6.586314,20.853764
1404363,id3862406,1.0,2016-05-11 02:49:00,2016-05-11 03:07:27,1.0,-73.983856,40.749092,-73.940125,40.835163,N,1107.0,Manhattan,fhv,no charge,10.254318,33.347374


In [20]:
# Simulate multi-collinearity by generating trip_distance_miles
df_dirty['trip_distance_miles'] = df_dirty['distance'] * 0.621371

# Simulate multi-collinearity by generating trip_duration_minutes
df_dirty['trip_duration_minutes'] = df_dirty['trip_duration'] / 60

In [23]:
# Simulate multi-collinearity with a linear combination of 3 random variables from the numerical columns
random_cols = np.random.choice(['passenger_count', 'trip_duration', 'distance', 'speed', 'trip_duration_minutes', 'trip_distance_miles'], 3, replace=False)
df_dirty['LC_sim'] = df_dirty[random_cols[0]] + 0.34 * df_dirty[random_cols[1]] + 0.12 * df_dirty[random_cols[2]]/3.2

# Save the dirty dataset
df_dirty.to_csv('./nyc-taxi-trip-duration/train_dirty.csv', index=False)

df_dirty.sample(3)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,nyc_borough,taxi_type,payment_type,distance,speed,trip_distance_miles,trip_duration_minutes,LC_sim
104623,id3009824,1.0,2016-05-30 18:01:56,2016-05-30 18:11:55,3.0,-73.992188,40.723923,-73.976303,40.725338,N,599.0,Queens,yellow,NaN,1.347789,8.100237,0.837477,9.983333,26.694311
1203065,id3864038,2.0,2016-01-12 00:29:28,2016-01-12 00:59:38,2.0,-73.776726,40.645329,-73.986092,40.758202,N,1810.0,Brooklyn,fhv,NaN,21.656866,43.074430,13.456949,30.166667,91.588615
215359,id0132066,2.0,2016-04-03 18:04:15,2016-04-03 18:22:23,1.0,-73.863525,40.769775,-73.964134,40.755394,N,1088.0,Bronx,green,cash,8.622997,28.531977,5.358081,18.133333,52.323414
